## Nettoyage de la base allocine

In [174]:
import pandas as pd 
import numpy as np 
from datetime import datetime
from dateutil import parser 

In [175]:
df_movies= pd.read_csv("df_film_ac.csv")

In [176]:
print(df_movies.dtypes)
#df_movies.isnull().sum()
#df_movies["date"].value_counts()

Titre original           object
Note press               object
Notes spectateur         object
Critiques presse         object
Critiques spectateurs    object
id allocine               int64
Box office               object
Budget                   object
date                     object
duree                    object
dtype: object


In [177]:
#modifier les types numériques 
df_movies['Box office'] = df_movies['Box office'].str.replace(' ', '').astype(float)
df_movies['Note press'] = df_movies['Note press'].str.replace(',', '.').astype(float)
df_movies['Notes spectateur'] = df_movies['Notes spectateur'].replace("--", np.nan)
df_movies['Notes spectateur'] = df_movies['Notes spectateur'].str.replace(',', '.').astype(float)

In [178]:
# Fonction pour convertir le mois en anglais
def french_to_english_month(month_french):
    months_mapping = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    return months_mapping.get(month_french.lower(), month_french)

# Remplacer les chaînes "nan" par des valeurs NaN
df_movies['date'] = df_movies['date'].replace('nan', np.nan)

# Appliquer la fonction pour convertir le mois en anglais
df_movies['date'] = df_movies['date'].apply(lambda x: ' '.join([french_to_english_month(word) for word in str(x).split()]) if pd.notna(x) else np.nan)

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_movies['date'] = df_movies['date'].apply(lambda x: parser.parse(x, dayfirst=True) if isinstance(x, str) else x)



In [179]:
# Fonction pour convertir la durée en minutes
def convert_duration(duration_str):
    if isinstance(duration_str, str):
        # Supprimer les espaces et diviser la chaîne en parties
        parts = duration_str.replace(' ', '').split('h')

        # Vérifier la présence des heures et des minutes
        if len(parts) == 2:
            hours = int(parts[0])
            minutes = 0 if 'min' not in parts[1] else int(parts[1].replace('min', ''))
            
            # Calculer la durée totale en minutes
            total_minutes = hours * 60 + minutes
            
            return int(total_minutes)
    
    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return float('nan')

# Appliquer la fonction de conversion
df_movies['duree_minutes'] = df_movies['duree'].apply(convert_duration)


In [180]:
#Extraire le nombre de critique de la presse 

def extract_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Utiliser isdigit() pour extraire uniquement les chiffres
        return np.nan if not critiques_str.split()[0].isdigit() else int(critiques_str.split()[0])
    else:
        return np.nan

# Appliquer la fonction de conversion
df_movies['Nombre_de_critiques_presse'] = df_movies['Critiques presse'].apply(extract_critiques_count)


In [181]:
#critiques et votes spectateurs 

# Fonction pour extraire le nombre de notes et le nombre de critiques
def extract_notes_and_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Trouver les nombres dans la chaîne
        numbers = [int(word) for word in critiques_str.split() if word.isdigit()]

        # Extraire le nombre de notes et de critiques en fonction de la longueur de la liste "numbers"
        if len(numbers) == 1:
            return numbers[0], np.nan
        elif len(numbers) == 2:
            return numbers[0], numbers[1]

    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return np.nan, np.nan

# Appliquer la fonction de conversion
df_movies[['Nombre_de_notes_spectateurs', 'Nombre_de_critiques_spectateurs']] = df_movies['Critiques spectateurs'].apply(extract_notes_and_critiques_count).apply(pd.Series)

In [182]:
df_movies.head()

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,31 critiques,80471 notes dont 4605 critiques,132039,127770000.0,80 000 000 $,2010-02-24,2h 17min,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,24 critiques,110095 notes dont 7212 critiques,143692,290948208.0,160 000 000 $,2010-07-21,2h 28min,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,20 critiques,52676 notes dont 2887 critiques,126693,291377000.0,150 000 000 $,2010-11-24,2h 26min,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,22 critiques,26730 notes dont 2133 critiques,126678,89981000.0,200 000 000 $,2010-05-26,2h 06min,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,20 critiques,10503 notes dont 1144 critiques,128955,92524000.0,80 000 000 $,2010-01-20,1h 49min,109.0,20.0,10503.0,1144.0


In [183]:
print(df_movies.dtypes)

Titre original                             object
Note press                                float64
Notes spectateur                          float64
Critiques presse                           object
Critiques spectateurs                      object
id allocine                                 int64
Box office                                float64
Budget                                     object
date                               datetime64[ns]
duree                                      object
duree_minutes                             float64
Nombre_de_critiques_presse                float64
Nombre_de_notes_spectateurs               float64
Nombre_de_critiques_spectateurs           float64
dtype: object


In [184]:
#on retire les colonnes que nous utilisons pas 
colonnes_a_retirer = ["Critiques presse","Critiques spectateurs","duree","Budget"]
df_movies_clean=df_movies.drop(colonnes_a_retirer,axis=1)

In [185]:
df_movies_clean.head()

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,20.0,10503.0,1144.0


In [350]:
#retrait de tous les films sans critique spectateurs
#on garde que les films avec au moins plus de 5 commentaires

df_film_avec_critiques = df_movies_clean.dropna(subset=['Nombre_de_critiques_spectateurs'])
df_film_avec_critiques = df_film_avec_critiques[df_film_avec_critiques['Nombre_de_critiques_spectateurs'] >= 5]
df_film_avec_critiques


,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,20.0,10503.0,1144.0
...,...,...,...,...,...,...,...,...,...,...
11101,The Gateway,NaN,2.3,273584,NaN,2022-03-16,91.0,NaN,40.0,6.0
11102,"Batman: The Long Halloween, Part Two",NaN,3.7,292641,NaN,2021-08-04,84.0,NaN,114.0,7.0
11104,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,NaN,50.0,6.0
11117,Black As Night,NaN,1.6,287632,NaN,2021-10-01,87.0,NaN,53.0,6.0


In [129]:
df_film_avec_critiques.to_csv('df_film_avec_critiques.csv', index=False)


## Nettoyage de la base TMDB

In [187]:
#engine=python permet de regler les problemes de compatibilité

df_tmdb= pd.read_csv("df_tmdb.csv", engine="python")
print(df_tmdb.dtypes)

adult                     object
backdrop_path             object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                       float64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
success                   object
status_code              float64
status_message            object
dtype: object


In [188]:
noms_variables = df_tmdb.columns.tolist()
print(noms_variables)

#for i in noms_variables:
    #print(i,":",df_tmdb[i].isnull().sum())

#on observe que les trois dernières colonnes peuvent être supprimés de la dataframe 
#trop de valeurs manquantes

colonnes_a_supprimer = ["success", "status_code", "status_message"]
df_tmdb_clean = df_tmdb.drop(columns=colonnes_a_supprimer)


['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'success', 'status_code', 'status_message']


In [189]:
#changement format release date 
# Supposons que df_tmdb soit votre DataFrame
df_tmdb_clean['release_date'] = pd.to_datetime(df_tmdb['release_date'], errors="coerce")
df_tmdb_clean["release_date"].value_counts()

release_date
2009-01-01    75
2011-01-01    71
2008-01-01    70
2014-01-01    68
2007-01-01    68
              ..
2021-08-21     1
1998-06-09     1
1950-05-19     1
1980-03-28     1
1983-09-16     1
Name: count, Length: 12586, dtype: int64

#### on garde uniquement les films qui sont dans la base allocine avec critique 

In [190]:
ma_liste = df_film_avec_critiques['Titre original'].tolist()
len(ma_liste)

4194

## Merging des deux bases

In [191]:
#séparation du mois et de l'année de sortie dans les deux bases

# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_film_avec_critiques['mois_sortie'] = df_film_avec_critiques['date'].dt.month
df_film_avec_critiques['annee_sortie'] = df_film_avec_critiques['date'].dt.year

df_film_avec_critiques.head()

# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_tmdb_clean['mois_sortie'] = df_tmdb_clean['release_date'].dt.month
df_tmdb_clean['annee_sortie'] = df_tmdb_clean['release_date'].dt.year

df_tmdb_clean.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,mois_sortie,annee_sortie
0,False,/ecvy2kMxsJ60ej52beZ0F8EOGkL.jpg,NaN,80000000.0,"Drama, Thriller, Mystery",http://www.shutterisland.com/,11324.0,tt1130884,en,Shutter Island,...,138.0,"English, Deutsch",Released,Some places never let you go.,Shutter Island,False,8.201,22634.0,2.0,2010.0
1,False,NaN,NaN,0.0,Documentary,NaN,1157141.0,tt5445282,en,Shutter Island: Into the Lighthouse,...,21.0,English,Released,NaN,Shutter Island: Into the Lighthouse,False,0.000,0.0,6.0,2010.0
2,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,NaN,160000000.0,"Action, Science Fiction, Adventure",https://www.warnerbros.com/movies/inception,27205.0,tt1375666,en,Inception,...,148.0,"English, Français, 日本語, Kiswahili",Released,Your mind is the scene of the crime.,Inception,False,8.366,34886.0,7.0,2010.0
3,False,/JeGkRdNsOuMrgwBdtB0hp763MU.jpg,El Crack Collection,0.0,"Drama, Thriller",https://filmaxinternationalsales.com/film/the-...,613092.0,tt6793710,es,El crack cero,...,117.0,Español,Released,The prequel to the cult film El Crack,The Crack: Inception,False,6.700,36.0,10.0,2019.0
4,False,NaN,NaN,0.0,"Animation, Action, Thriller, Science Fiction",NaN,64956.0,tt5295894,en,Inception: The Cobol Job,...,14.0,English,Released,NaN,Inception: The Cobol Job,False,7.300,290.0,12.0,2010.0


In [361]:
df_film_avec_critiques['titre_normalise'] = df_film_avec_critiques['Titre original'].str.replace(r'\W', '', regex=True).str.lower()
df_tmdb_clean['titre_normalise'] = df_tmdb_clean['original_title'].str.replace(r'\W', '', regex=True).str.lower()

In [229]:
ma_liste=df_film_avec_critiques["titre_normalise"].tolist()
len(ma_liste)
ma_liste_bis=df_tmdb_clean["titre_normalise"].tolist()
len(ma_liste_bis)

34379

In [215]:
#nous commencons par prendre les elements qui se ressemblent dans les deux listes
nouvelle_liste = [element for element in ma_liste_bis if element in ma_liste]
len(nouvelle_liste)

11542

In [351]:
#on remarque à ce niveau que certains films apparaissent plusieurs fois dans nouvelle_liste du fait que de nombreux films sont sortis 
#sous le meme nom 

#je commence par garder dans la base df_tmdb_clean que les films qui sont dans nouvelle_liste
df_filtre = df_tmdb_clean[df_tmdb_clean['titre_normalise'].isin(nouvelle_liste)]
len(df_filtre)

#je trie ensuite la base par ordre alphabetique des films et decroissants des dates
df_filtre_trie = df_filtre.sort_values(by=['titre_normalise', 'release_date'], ascending=[True, False])
df_filtre_trie = df_filtre_trie.reset_index(drop=True)

#conservation des films sorties entre 2010 et 2021 (et valeurs manquantes aussi)
df_filtre_trie = df_filtre_trie[(df_filtre_trie['annee_sortie'] >= 2010) & (df_filtre_trie['annee_sortie'] <= 2021) | df_filtre_trie['annee_sortie'].isna()]
df_filtre_trie = df_filtre_trie.reset_index(drop=True)

#on retire tous les doublons parfaits de la base 
df_filtre_trie = df_filtre_trie.drop_duplicates(keep='first')
df_filtre_trie = df_filtre_trie.reset_index(drop=True)

df_filtre_trie.to_csv("df_tri.csv", index=False)

In [228]:
len(df_filtre_trie)

7312

In [248]:
from collections import Counter

liste_nouv = df_filtre_trie["titre_normalise"].tolist()
# Compter les occurrences dans nouvelle_liste
comptages = Counter(liste_nouv)

# Trouver les éléments qui apparaissent plus d'une fois
elements_en_double = [element for element, count in comptages.items() if count > 1]

#print("Éléments en plusieurs fois :", elements_en_double)
len(elements_en_double)

#df_test=df_filtre_trie[df_filtre_trie["titre_normalise"].isin(elements_en_double)]
#df_test = df_test.reset_index(drop=True)

1120

In [244]:
df_test=df_filtre_trie[df_filtre_trie["titre_normalise"].isin(elements_en_double)]
df_test = df_test.reset_index(drop=True)
df_test=df_test[["title","runtime","release_date"]]
df_test.to_csv("df_test.csv", index=False)


In [263]:
len(df_test)

4478

In [249]:
df_test_bis = df_film_avec_critiques.sort_values(by=['titre_normalise', 'date'], ascending=[True, False])
df_test_bis = df_test_bis.reset_index(drop=True)
df_test_bis=df_test_bis[["Titre original","duree_minutes","date"]]
df_test_bis.to_csv("df_test_bis.csv", index=False)

In [247]:
len(df_test_bis)

1158

In [250]:
df_film_avec_critiques["duree_minutes"].isnull().sum()

47

In [257]:
import pandas as pd
import numpy as np

# Fusion des deux DataFrames sur la colonne 'titre_original' et 'titre_normalise'
df_fusion = pd.merge(df_film_avec_critiques, df_filtre_trie, left_on='titre_normalise', right_on='titre_normalise')

# Ajouter une condition pour filtrer en fonction de la différence de durée ou si une des colonnes a une valeur NaN
df_resultat = df_fusion[(abs(df_fusion['duree_minutes'] - df_fusion['runtime']) <= 3) | (df_fusion['duree_minutes'].isna()) | (df_fusion['runtime'].isna())]

# Réinitialiser l'index après le filtrage
df_resultat = df_resultat.reset_index(drop=True)

# Afficher le DataFrame résultant
len(df_resultat)


#


3520

In [258]:
df_resultat.to_csv("df_merge.csv", index=False)


In [282]:
liste_id = df_filtre_trie["id"].tolist()
liste_id_entiers = [int(nombre) for nombre in liste_id]
len(liste_id_entiers)
#liste_id_tronque=[liste_id_entiers[0],liste_id_entiers[1],liste_id_entiers[2]]
#liste_id_tronque

7312

In [261]:
%%time
import requests

movie_id = "11324"
api_key = "d1d1413d8379729633d60e9f5cc4a730"

# Envoyer une requête GET à l'API TMDb pour obtenir les crédits du film
response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}')

# Vérifier si la requête a réussi (code 200)
if response.status_code == 200:
    # Extraire le contenu JSON de la réponse
    json_data = response.json()

    # Filtrer la liste des membres de l'équipe pour ne conserver que les réalisateurs
    directors = [member for member in json_data.get('crew', []) if member.get('job') == 'Director']

    # Afficher la liste des réalisateurs
    #print(directors)
    # Supposons que data contient la liste des réalisateurs
    realisateur = [director['name'] for director in directors]
# Afficher la liste des noms des réalisateurs
    print(realisateur)

else:
    # En cas d'échec de la requête, afficher le code d'erreur
    print(f"Error: {response.status_code}")


['Martin Scorsese']


In [283]:
%%time

def get_directors_for_movies(movie_ids, api_key):
    directors_dict = {}

    for movie_id in movie_ids:
        # Envoyer une requête GET à l'API TMDb pour obtenir les crédits du film
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}')

        # Vérifier si la requête a réussi (code 200)
        if response.status_code == 200:
            # Extraire le contenu JSON de la réponse
            json_data = response.json()

            # Filtrer la liste des membres de l'équipe pour ne conserver que les réalisateurs
            directors = [member.get('name') for member in json_data.get('crew', []) if member.get('job') == 'Director']

            # Ajouter l'association identifiant-réalisateur au dictionnaire
            directors_dict[movie_id] = directors if directors else None
        else:
            # En cas d'échec de la requête, ajouter une valeur manquante au dictionnaire
            directors_dict[movie_id] = None

    return directors_dict

# Exemple d'utilisation avec une liste d'identifiants de films
api_key = "d1d1413d8379729633d60e9f5cc4a730"

# Appeler la fonction pour obtenir le dictionnaire des réalisateurs
directors_result = get_directors_for_movies(liste_id_entiers, api_key)

# Afficher le résultat
len(directors_result)


KeyboardInterrupt: 

In [284]:
len(directors_result)

7296

In [352]:

# Ajouter une colonne 'nom_du_realisateur' en utilisant le dictionnaire
df_filtre_trie['nom_du_realisateur'] = df_filtre_trie['id'].map(directors_result)

# Afficher le DataFrame résultant
df_filtre_trie.head()
df_filtre_trie.to_csv("df_filtre_trie.csv", index=False)


###### note : on peut pas merge pcq pb au niveau des dates de sorties. On devrait peut etre faire avec la duree du film pour merge (cf idee + ou - 1 minute) 

In [295]:
%%time

def get_director_name(url):
    # Envoyer une requête GET à l'URL de la page Allociné
    response = requests.get(url)

    # Vérifier si la requête a réussi (code 200)
    if response.status_code == 200:
        # Extraire le contenu HTML de la réponse
        html_content = response.content

        # Utiliser BeautifulSoup pour parcourir le HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Trouver la balise contenant le nom du réalisateur
        director_tag = soup.find('div', class_='meta-body-direction')

        # Extraire le texte entre les balises
        director_name = director_tag.text.strip() if director_tag else None

        return director_name

    else:
        # En cas d'échec de la requête, retourner une valeur manquante
        return None

def get_director_names_from_urls(identifiants):
    director_names = {}

    for identifiant in identifiants:
        # Construire l'URL à partir de l'identifiant
        url = f'https://www.allocine.fr/film/fichefilm_gen_cfilm={identifiant}.html'

        # Appeler la fonction pour obtenir le nom du réalisateur
        nom_realisateur = get_director_name(url)

        # Ajouter le nom du réalisateur au dictionnaire
        director_names[identifiant] = nom_realisateur

    return director_names

liste_identifiants = df_film_avec_critiques["id allocine"].tolist()
# Exemple d'utilisation avec une liste d'identifiants
resultat = get_director_names_from_urls(liste_identifiants)

# Afficher le résultat
#for identifiant, nom_realisateur in resultat.items():
    #print(f"Film {identifiant}: Réalisateur {nom_realisateur}")

print(len(resultat))
print(resultat)

4194
{132039: 'De\nMartin Scorsese', 143692: 'De\nChristopher Nolan', 126693: 'De\nDavid Yates', 126678: 'De\nMike Newell', 128955: 'De\nAlbert Hughes,\nAllen Hughes', 126127: 'De\nRidley Scott', 136370: 'De\nEdgar Wright', 140620: 'De\nSusanna White', 123534: 'De\nChris Sanders,\nDean DeBlois', 125828: 'De\nDarren Aronofsky', 147912: 'De\nDavid Fincher', 135523: 'De\nByron Howard,\nNathan Greno', 128105: 'De\nChris Columbus', 183144: 'De\nSteven R. Monroe', 145368: 'De\nJames Mangold', 126386: 'De\nDavid O. Russell', 174939: 'De\nDanny Boyle', 132663: 'De\nTim Burton', 131494: 'De\nDerek Cianfrance', 140603: 'De\nHarald Zwart', 121807: 'De\nRyan Murphy', 138752: 'De\nFlorian Henckel von Donnersmarck', 140633: 'De\nClint Eastwood', 145331: 'De\nAllen Coulter', 82376: 'De\nBen Affleck', 147203: 'De\nEdward Zwick', 182603: 'De\nJames Wan', 129368: 'De\nSylvain White', 125433: 'De\nM. Night Shyamalan', 134196: 'De\nScott Stewart (XX)', 140458: 'De\nSylvester Stallone', 146402: 'De\nGeorge

In [353]:
# Ajouter une colonne 'nom_du_realisateur' en utilisant le dictionnaire
df_film_avec_critiques['nom_du_realisateur'] = df_film_avec_critiques['id allocine'].map(resultat)

# Afficher le DataFrame résultant
df_film_avec_critiques.head()
df_film_avec_critiques.to_csv("df_film_avec_critiques.csv", index=False)


In [ ]:
#cdf_filtre_trie
#df_film_avec_critiques

In [354]:
#df_film_avec_critiques["nom_du_realisateur"][0]
df_film_avec_critiques['nom_du_realisateur'] = df_film_avec_critiques['nom_du_realisateur'].str.replace('De\n', '')

In [355]:
#ne pas mettre dans la meme cellule qu'avant

df_film_avec_critiques['nom_du_realisateur'] = df_film_avec_critiques['nom_du_realisateur'].str.replace('\n', '')
df_film_avec_critiques.head()

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,nom_du_realisateur
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0,Martin Scorsese
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0,Christopher Nolan
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0,David Yates
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,22.0,26730.0,2133.0,Mike Newell
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,20.0,10503.0,1144.0,"Albert Hughes,Allen Hughes"


In [356]:
df_filtre_trie['nom_du_realisateur'] = df_filtre_trie['nom_du_realisateur'].apply(lambda x: x[0] if x else None)
df_filtre_trie.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,status,tagline,title,video,vote_average,vote_count,mois_sortie,annee_sortie,titre_normalise,nom_du_realisateur
0,False,NaN,NaN,0.0,Science Fiction,NaN,293082.0,tt3530690,en,"10,000 Days",...,Released,NaN,"10,000 Days",False,2.800,15.0,11.0,2014.0,10000days,Eric Small
1,False,/wkhtdTSQKAIoOD7f6zD9BXGsr7f.jpg,NaN,500000.0,"Action, Science Fiction",NaN,182228.0,tt2538128,en,100 Degrees Below Zero,...,Released,Cold as Hell,100 Degrees Below Zero,False,4.400,112.0,3.0,2013.0,100degreesbelowzero,R.D. Braunstein
2,False,/5uLqjVQlKNR4MbOeuheaRihPxAx.jpg,NaN,15000000.0,"Thriller, Science Fiction, Drama, Horror",http://www.10cloverfieldlane.com/,333371.0,tt1179933,en,10 Cloverfield Lane,...,Released,Monsters come in many forms.,10 Cloverfield Lane,False,6.984,7456.0,3.0,2016.0,10cloverfieldlane,Dan Trachtenberg
3,False,/sbqm63cOAG5eB9WsUR1hY1zpkTH.jpg,NaN,0.0,"Crime, Action, Thriller, Mystery",NaN,552865.0,tt8809652,en,10 Minutes Gone,...,Released,Keep your enemies close,10 Minutes Gone,False,5.358,204.0,9.0,2019.0,10minutesgone,Brian A. Miller
4,False,/9QWBstBOwv36cuv0hK4esNyFnvo.jpg,NaN,0.0,Thriller,NaN,505177.0,tt6210996,en,10 x 10,...,Released,There are some secrets we cannot escape,10x10,False,5.200,508.0,4.0,2018.0,10x10,Suzi Ewing


In [375]:
import re

def remove_accents(input_str):
    accents = {
        'a': '[aáàâäãå]',
        'e': '[eéèêë]',
        'i': '[iíìîï]',
        'o': '[oóòôöõ]',
        'u': '[uúùûü]',
        'c': '[cç]',
        'n': '[nñ]'
    }
    for char, pattern in accents.items():
        input_str = re.sub(pattern, char, input_str)
    return input_str

# Appliquer la fonction remove_accents à la colonne nom_du_realisateur
df_filtre_trie['nom_du_realisateur_bis'] = df_filtre_trie['nom_du_realisateur'].apply(lambda x: remove_accents(x) if pd.notnull(x) else x)
df_film_avec_critiques['nom_du_realisateur_bis'] = df_film_avec_critiques['nom_du_realisateur'].apply(lambda x: remove_accents(x) if pd.notnull(x) else x)


In [371]:
df_film_avec_critiques["nom_du_realisateur_bis"][11]

'roelreiné'

In [376]:
df_film_avec_critiques['nom_du_realisateur_bis'] = df_film_avec_critiques['nom_du_realisateur_bis'].str.replace(r'\W', '', regex=True).str.lower()
#df_film_avec_critiques.head()
df_filtre_trie['nom_du_realisateur_bis'] = df_filtre_trie['nom_du_realisateur_bis'].str.replace(r'\W', '', regex=True).str.lower()
df_filtre_trie.head()

#df_film_avec_critiques = df_film_avec_critiques.sort_values(by=['titre_normalise', 'date'], ascending=[True, False])


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,tagline,title,video,vote_average,vote_count,mois_sortie,annee_sortie,titre_normalise,nom_du_realisateur,nom_du_realisateur_bis
0,False,NaN,NaN,0.0,Science Fiction,NaN,293082.0,tt3530690,en,"10,000 Days",...,NaN,"10,000 Days",False,2.800,15.0,11.0,2014.0,10000days,Eric Small,ericsmall
1,False,/wkhtdTSQKAIoOD7f6zD9BXGsr7f.jpg,NaN,500000.0,"Action, Science Fiction",NaN,182228.0,tt2538128,en,100 Degrees Below Zero,...,Cold as Hell,100 Degrees Below Zero,False,4.400,112.0,3.0,2013.0,100degreesbelowzero,R.D. Braunstein,rdbraunstein
2,False,/5uLqjVQlKNR4MbOeuheaRihPxAx.jpg,NaN,15000000.0,"Thriller, Science Fiction, Drama, Horror",http://www.10cloverfieldlane.com/,333371.0,tt1179933,en,10 Cloverfield Lane,...,Monsters come in many forms.,10 Cloverfield Lane,False,6.984,7456.0,3.0,2016.0,10cloverfieldlane,Dan Trachtenberg,dantrachtenberg
3,False,/sbqm63cOAG5eB9WsUR1hY1zpkTH.jpg,NaN,0.0,"Crime, Action, Thriller, Mystery",NaN,552865.0,tt8809652,en,10 Minutes Gone,...,Keep your enemies close,10 Minutes Gone,False,5.358,204.0,9.0,2019.0,10minutesgone,Brian A. Miller,brianamiller
4,False,/9QWBstBOwv36cuv0hK4esNyFnvo.jpg,NaN,0.0,Thriller,NaN,505177.0,tt6210996,en,10 x 10,...,There are some secrets we cannot escape,10x10,False,5.200,508.0,4.0,2018.0,10x10,Suzi Ewing,suziewing


In [377]:
df_film_avec_critiques["nom_du_realisateur_bis"][11]

'roelreine'

In [364]:
df_film_avec_critiques = df_film_avec_critiques.sort_values(by=['titre_normalise', 'date'], ascending=[True, False])
df_film_avec_critiques = df_film_avec_critiques.reset_index(drop=True)
df_film_avec_critiques.head(12)

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,nom_du_realisateur,nom_du_realisateur_bis,titre_normalise
0,+1,NaN,2.6,223482,NaN,NaT,100.0,NaN,30.0,8.0,Dennis Iliadis,dennisiliadis,1
1,10 000 days,NaN,1.5,239331,NaN,NaT,91.0,NaN,32.0,7.0,Eric Small,ericsmall,10000days
2,100 Degrees Below Zero,NaN,1.2,221267,NaN,2015-04-21,89.0,NaN,110.0,51.0,R.D. Braunstein,rdbraunstein,100degreesbelowzero
3,10 Cloverfield Lane,3.6,3.6,231144,67880730.0,2016-03-16,103.0,20.0,9399.0,735.0,Dan Trachtenberg,dantrachtenberg,10cloverfieldlane
4,10 Minutes Gone,NaN,1.5,266802,NaN,2020-01-10,96.0,NaN,170.0,23.0,Brian A. Miller,brianamiller,10minutesgone
5,10x10,NaN,2.0,251534,NaN,2018-11-29,87.0,NaN,447.0,54.0,Suzi Ewing,suziewing,10x10
6,10 Years,NaN,2.3,189144,NaN,2017-06-02,97.0,NaN,306.0,12.0,Jamie Linden,jamielinden,10years
7,11-11-11,NaN,1.4,184901,NaN,2012-10-24,90.0,NaN,191.0,50.0,Darren Lynn Bousman,darrenlynnbousman,111111
8,127 Hours,3.2,3.9,174939,16845896.0,2011-02-23,94.0,28.0,24166.0,1574.0,Danny Boyle,dannyboyle,127hours
9,12 Mighty Orphans,NaN,3.5,275746,3326335.0,2021-08-31,118.0,NaN,87.0,10.0,Ty Roberts,tyroberts,12mightyorphans


In [381]:
#conserve uniquement les correspondances
df_merged = pd.merge(df_film_avec_critiques, df_filtre_trie, left_on=['titre_normalise', 'nom_du_realisateur_bis'], right_on=['titre_normalise', 'nom_du_realisateur_bis'])
len(df_merged)

df_merged.to_csv("df_merged.csv", index=False)

#conserve les correspondances et tout du cote df_film_avec_critiques
df_merged_bis= pd.merge(df_film_avec_critiques, df_filtre_trie, on=['titre_normalise', 'nom_du_realisateur_bis'], how='left')
df_merged_bis.to_csv("df_merged_bis.csv", index=False)


In [382]:
len(df_merged_bis)

4231

In [378]:
new_df = pd.DataFrame({
    'col1_df1': df_film_avec_critiques['titre_normalise'],
    'col2_df1': df_film_avec_critiques['nom_du_realisateur_bis'],
    'col3_df2': df_filtre_trie['titre_normalise'],
    'col4_df2': df_filtre_trie['nom_du_realisateur_bis']
})

new_df.to_csv("new_df.csv", index=False)

